## Чат-бот - составитель упражнений по юридическому нидерландскому языку

Чат-бот может выполнять две задачи:
*   получать от пользователя текст решения Верховного суда Нидерландов по гражданским делам (или его фрагмент, отдельные предложения) и выдавать тот же текст, но с пропусками вместо глаголов и списком этих глаголов в начальной форме;
*   выдавать пользователю по его запросу текст решения Верховного суда Нидерландов по гражданским делам со списком его ключевых слов, при этом пользователь может выбрать объем текста и степень его лексического разнообразия

# 1. Загрузка библиотек

In [ ]:
!pip install pytelegrambotapi

In [ ]:
!pip install stanza
import stanza
stanza.download('nl')
nlp = stanza.Pipeline(lang = 'nl')

In [ ]:
import re

In [ ]:
!pip install nltk
import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download('stopwords')
nl_stop_words = stopwords.words('dutch')

In [ ]:
from collections import Counter
import os

In [ ]:
!pip install pandas
import pandas as pd

In [ ]:
import telebot
from telebot import types

# 2. Ячейка для ввода идентификатора бота, созданного в Телеграме

In [ ]:
Dutch_bot = 'КОД'

# 3. Функции

## 3.1. Get_verbs
Функция нужна для первой задачи чат-бота. Она предобрабатывает текст, полученный от пользователя, отправляет преодобработанный текст в Stanza, ищет в нем глаголы и в результате выдает текст с пропущенными глаголами и список этих глаголов в начальной форме

In [ ]:
def get_verbs(text):
  # чистка текста
  # в основном удаляются слипшиеся буквы и цифры
  # также после "naar" удаляется двоеточие, чтобы Stanza не считала часть предложения до двоеточия самостоятельным фрагментом текста
  # это важно для текстов судебных решений Верховного суда Нидерландов, потому что в них используется устойчивая конструкция:
  # "...verwijst de Hoge Raad naar: het vonnis..." ("...Верховный суд отсылает к: решению...")
  # если двоеточие не удалить, Stanza будет считать "naar" не предлогом, а составной частью глагола,
  # что будет приводить к ошибке при приведении глаголов в начальную форму
  text_clean = re.sub(r'naar:', 'naar', text)
  text_clean = re.sub(r'(?<=\d)(?=[A-Z])', '. ', text_clean)
  text_clean = re.sub(r'(?<=\d\.\d)\s', '. ', text_clean)
  text_clean = re.sub(r'(?<=[A-Za-z])(?=\d+)', ' ', text_clean)

  # в словах, написанных полностью заглавными буквами, оставляем заглавной только первую букву
  # иначе Stanza их некорректно лемматизирует
  naive_tokens = []
  for i in text_clean.split():
    if i.isupper():
      naive_tokens.append(f'{i[0]}{i[1:].lower()}')
    else:
      naive_tokens.append(i)
  text_clean = ' '.join(naive_tokens)

  # направляем чистый текст в Stanza
  text_clean_stanza = nlp(text_clean)

  # собираем в списки глаголы, их отделяемые приставки и текст с пропусками
  # счетчик вводим для подсчета предложений, поскольку в Stanza нет встроенного id предложений
  verb_lemmas = []
  verb_texts = []
  verb_ids = []

  id_sentences_1 = []
  id_sentences_2 = []
  count = 0

  text_with_gaps = []

  part_texts = []
  head_part_texts = []

  for sentence in text_clean_stanza.sentences:
    for word in sentence.words:
      if (word.upos == "VERB") and (word.deprel != 'amod') and (word.deprel != 'obl:arg') and  (word.deprel != 'obl') and (word.deprel != 'obj'):
        verb_lemmas.append(word.lemma)
        verb_texts.append(word.text)
        verb_ids.append(word.id)
        id_sentences_1.append(str(count))
        text_with_gaps.append('_____')
      elif (word.upos == "ADP") and (word.deprel == "compound:prt") and (word.xpos == "VZ|fin"):
        part_texts.append(word.text)
        head_part_texts.append(word.head)
        id_sentences_2.append(str(count))
      else:
        text_with_gaps.append(word.text)
    count = count + 1

  # немного чистим и структурируем текст с пропусками для выдачи чат-ботом
  # некоторые символы написала юникодом просто для интереса, чтобы обойтись без экранирования
  final_text = ' '.join(text_with_gaps)
  final_text = re.sub(r'\s(?=[\.,!?:;\u0029])', '', final_text)
  final_text = re.sub(r'(?<=\u0028)\s', '', final_text)
  final_text = re.sub(r'\s(?=\d\. Beoordeling van het middel|\d\. Procesverloop|\d\. Uitgangspunten en feiten|\d\. Beslissing)', '\n\n', final_text)
  final_text = re.sub(r'((?<=\d\. Beoordeling van het middel)|(?<=\d\. Procesverloop))|(?<=\d\. Uitgangspunten en feiten)|(?<=\d\. Beslissing)\s', '\n', final_text)
  final_text = re.sub(r'\s(?=\d\. Uitgangspunten)', '\n\n', final_text)
  final_text = re.sub(r'(?<=\d\. Uitgangspunten)\s', '\n', final_text)
  final_text = re.sub(r'^Uitspraak ', 'Uitspraak\n', final_text)
  final_text = re.sub(r'\nHoge Raad Der Nederlanden ', '\nHoge Raad Der Nederlanden\n', final_text)
  final_text = re.sub(r'\nCiviele Kamer ', '\nCiviele Kamer\n', final_text)
  final_text = re.sub(r'Arrest (?=In de zaak van)', '\nArrest\n', final_text)
  final_text = re.sub(r'Beschikking (?=In de zaak van)', '\nBeschikking\n', final_text)
  final_text = re.sub(r'\s(?=\d\.\d\. [A-Z])', '\n', final_text)
  final_text = re.sub(r'\s(?=\d\.\d\.\d\. [A-Z])', '\n', final_text)
  final_text = re.sub(r'\s(?=\u0028[ixv]+\u0029)', '\n', final_text)

  # собираем глаголы и отделяемые приставки глаголов в 2 дата-фрейма
  data_frame_verbs = pd.DataFrame([id_sentences_1, verb_lemmas, verb_texts, verb_ids]).T.rename(columns={0: "id_sentences", 1: "lemma", 2: "text", 3:"id"})
  data_frame_part = pd.DataFrame([id_sentences_2, part_texts, head_part_texts]).T.rename(columns={0: "id_sentences", 1: "text_part", 2:"id"})

  # сливаем 2 дата-фрейма в 1 по признаку id предложения и id head,
  # чтобы отделяемые приставки глаголов оказались рядом с глаголами, к которым они относятся
  merged_df = pd.merge(data_frame_verbs, data_frame_part, on=["id_sentences", "id"], how='outer')

  # глаголы без отделяемых приставок и глаголы, которые были в тексте уже вместе с приставками (инфинитивы, перфект),
  # собираем в simple_verbs
  df_with_nan = merged_df[merged_df['text_part'].isna()].copy()
  df_with_nan['lemma'] = df_with_nan['lemma'].apply(lambda x: re.sub(r'_', '', str(x)))
  simple_verbs = df_with_nan['lemma'].values.tolist()

  # с остальными глаголами сложнее
  # в glued_verbs_2 собираем склеенные глаголы с приставками
  # в glued_verbs тоже, но это список для тех глаголов, которые Stanza решила лемматизировать сразу до глагола с приставкой
  # поэтому сначала пришлось отрезать от леммы приставку, которую добавила сама Stanza,
  # и уже затем склеить глагол с приставкой из текста, чтобы не было задвоения приставок
  df_without_nan = merged_df[merged_df['text_part'].notna()].copy()

  filtered_df_1 = df_without_nan[df_without_nan['lemma'].str.contains('_', na=False)].copy()
  filtered_df_1['lemma'] = filtered_df_1['lemma'].apply(lambda x: re.sub(r'[a-z]+_', '', str(x)))
  filtered_df_1['lemma_full'] = filtered_df_1['text_part'] + filtered_df_1['lemma']
  glued_verbs = filtered_df_1['lemma_full'].values.tolist()

  filtered_df_2 = df_without_nan[~df_without_nan['lemma'].str.contains('_', na=False)].copy()
  filtered_df_2['lemma_full'] = filtered_df_2['text_part'] + filtered_df_2['lemma']
  glued_verbs_2 = filtered_df_2['lemma_full'].values.tolist()

  list_of_verbs = simple_verbs + glued_verbs + glued_verbs_2

  # некоторые глаголы используются в тексте несколько раз
  # для них сделаем выдачу в виде "verwijzen(3)"
  final_list_of_verbs = []
  for i in Counter(list_of_verbs):
    if Counter(list_of_verbs)[i] > 1:
      final_list_of_verbs.append(f'{i}({Counter(list_of_verbs)[i]})')
    else:
      final_list_of_verbs.append(f'{i}')

  # перемешаем глаголы с помощью set, чтобы они шли не в том порядке, в каком они идут в тексте
  final_list_of_verbs_mixed = list(set(final_list_of_verbs))

  # финальная выдача: "Вставьте глагол в нужной форме. <текст с пропусками> <список глаголов>"
  result_text_without_verbs = f'Vul de juiste vorm van het werkwoord in.\n\n {final_text}\n\n Werkwoorden: \n {", ".join(final_list_of_verbs_mixed)}'

  return result_text_without_verbs


## 3.2. Measure_ttr
Функция нужна для второй задачи чат-бота. Она рассчитывает порог лексического разнообразия, чтобы в дальнейшем по запросу пользователя могли выдаваться либо более, либо менее лексически разнообразные тексты судебных решений.

In [ ]:
def measure_ttr(min_ttr, max_ttr):
  threshold_ttr = min_ttr + round(max_ttr - min_ttr, 3) / 2
  return threshold_ttr

## 3.3. To_do
Функция также нужна для второй задачи чат-бота. На входе она принимает идентификатор чата, дата-фрейм с текстами судебных решений и словарь (будут введены далее), а выдает текст судебного решения, который нужно направить пользователю, и идентификатор судебного решения в базе Верховного суда Нидерландов (чтобы, например, пользователь потом мог посмотреть предысторию судебного разбирательства в низших инстанциях).

In [ ]:
def to_do(chat, data_frame, dictionary):
  # если пользователь ещё не запрашивал в чат-боте текст судебного решения, ему выдается текст из первой строки дата-фрейма
  if str(chat) not in dictionary:
    text_to_send = data_frame.iloc[0, 6]
    ecli_decision = data_frame.iloc[0, 0]
    dictionary[str(chat)] = 0

  # если пользователь уже запрашивал ранее тексты судебных решений,
  # то выдается следующий по счету в дата-фрейме текст судебного решения
  elif str(chat) in dictionary and dictionary[str(chat)] < (data_frame.shape[0]-1):
    dictionary[str(chat)] = dictionary[str(chat)] + 1
    k = dictionary[str(chat)]
    text_to_send = data_frame.iloc[k, 6]
    ecli_decision = data_frame.iloc[k, 0]

  # если тексты в дата-фрейме закончились, пользователю начинают направляться ранее выданные тексты,
  # при этом в первом повторном тексте делается отметка "повтор"
  elif str(chat) in dictionary and dictionary[str(chat)] == (data_frame.shape[0]-1):
    text_to_send = data_frame.iloc[0, 6]
    ecli_decision = '_ПОВТОР!_' + data_frame.iloc[0, 0]
    dictionary[str(chat)] = 0

  return text_to_send, ecli_decision

# 4. Чат-бот

In [ ]:
# словарь для первой задачи чат-бота
d = {}

# словарики для второй задачи чат-бота
dictionary_1 = {}
dictionary_2A = {}
dictionary_2B = {}
dictionary_3A = {}
dictionary_3B = {}

bot = telebot.TeleBot(Dutch_bot)

# стартовый диалог вместе с клавиатурой, где можно выбрать одну из задач чат-бота:
# удалить из текста, присланного пользователем, глаголы, и выдать текст с пропусками и список этих глаголов в начальной форме
# получить текст судебного решения Верхового суда Нидерландов (пользователь может выбрать размер текста и степень его лексического разнообразия)
@bot.message_handler(commands=["start"])
def start(message, res=False):
  markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
  item1=types.KeyboardButton("Удалить глаголы")
  item2=types.KeyboardButton("Получить текст судебного решения")
  markup.row(item1, item2)
  bot.send_message(message.chat.id, 'Добрый день! Я - помощник составителя упражнений по юридическому нидерландскому языку.')
  bot.send_message(message.chat.id, 'Я могу помочь: \n - удалить из текста судебного решения Верховного суда Нидерландов (из его фрагмента, отдельных предложений) глаголы, оставив вместо них пропуски и добавив после текста список этих глаголов (вперемешку) в начальной форме, \n - прислать текст решения Верховного суда Нидерландов со списком его ключевых слов (для чтения и обсуждения).')
  bot.send_message(message.chat.id, 'Что вы хотите сделать?', reply_markup=markup)

# следующий шаг - реакция на то, что укажет пользователь
@bot.message_handler(content_types=["text"])
def handle_text(message):
  if message.text == 'Удалить глаголы':
    bot.send_message(message.chat.id, 'Присылайте текст в формате .txt! Его загрузка может занять некоторое время. Дождитесь сообщения об успешной загрузке.')
    bot.register_next_step_handler(message, get_doc)

  elif message.text == 'Получить текст судебного решения':
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1=types.KeyboardButton("1")
    item2=types.KeyboardButton("2A")
    item3=types.KeyboardButton("2B")
    item4=types.KeyboardButton("3A")
    item5=types.KeyboardButton("3B")
    markup.row(item1, item2, item3)
    markup.row(item4, item5)
    bot.send_message(message.chat.id, 'Какой вам прислать текст? \n 1 - короткий (до 5 стр.), 2 - средний (до 15 стр.), 3 - длинный (больше 15 стр.), \n "A" - менее лексически разнообразный, "B" - более лексически разнообразный \n Если в банке документов закончатся файлы, бот пришлет старый файл с пометкой "повтор".', reply_markup=markup)
    bot.register_next_step_handler(message, send_decision)

# этот шаг запускается, если пользователь выбрал ранее "Удалить глаголы"
# в нем происходит загрузка текста, отправленного пользователем,
# его обработка и сохранение в словарь d
@bot.message_handler(content_types=["document"])
def get_doc(message):
  chat_id = message.chat.id
  file_info = bot.get_file(message.document.file_id)
  downloaded_file = bot.download_file(file_info.file_path)
  filename = message.document.file_name

  # в дальнейшем username прямо использоваться не будет для различения пользователей,
  # но в словарь его всё равно сохраняю чуть позже,
  # чтобы, если понадобится, проще можно было ориентироваться в словаре, не только по chat_id
  if message.from_user.username is not None:
    username = message.from_user.username
  elif message.from_user.last_name is not None:
    username = message.from_user.last_name
  else:
    username = message.from_user.first_name

  # подчищаем директорию и словарь, если пользователь не в первый раз обращается в чат-бот
  for i in os.listdir():
    if str(chat_id) in i:
      os.remove(i)

  if str(chat_id) in d:
    del d[str(chat_id)]

  # записываем файл пользователя с указанием идентификатора диалога
  with open(str(chat_id) + filename, 'wb') as new_file:
    new_file.write(downloaded_file)


  for i in os.listdir():
    if str(chat_id) in i and '.txt' in i:
      with open(i, 'r', encoding='utf-8') as input_file:
        initial_text = input_file.read()
      d[str(chat_id)] = ['Всё загрузилось!', username, initial_text, get_verbs(initial_text)]
    elif str(chat_id) in i and '.txt' not in i:
      d[str(chat_id)] = ['Файл должен быть в формате .txt! Пришлите, пожалуйста, файл в формате .txt.']
    # else:
    #   d[str(chat_id)] = ['Что-то пошло не так! Начните, пожалуйста, со /start.']

  markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
  item1=types.KeyboardButton("Обработать текст")
  markup.row(item1)
  bot.send_message(message.chat.id, f'{d[str(chat_id)][0]}')
  bot.send_message(message.chat.id, 'Если всё загрузилось - нажимайте на кнопку "Обработать текст". \n Если нет, загружайте другой файл либо вернитесь к /start.', reply_markup=markup)

  bot.register_next_step_handler(message, del_verbs)

# этот шаг запускается, если пользователь нажал "Обработать текст"
@bot.message_handler(content_types=["text"])
def del_verbs(message):
  if message.text == 'Обработать текст':

    chat_id = message.chat.id

    # подчищаем директорию
    for i in os.listdir():
      if str(chat_id) in i:
        os.remove(i)

    # направляем пользователю файл, в котором находится текст с пропусками глаголов и список этих глаголов в начальной форме
    with open(str(chat_id) + '_verbs', 'w', encoding='utf-8') as v:
      print(d[str(chat_id)][3], file = v)

    doc = open(str(chat_id) + '_verbs', 'r', encoding='utf-8')
    bot.send_document(message.chat.id, doc)
    doc.close()

  else:
    bot.send_message(message.chat.id, 'Что-то пошло не так! Начните, пожалуйста, со /start')

# этот шаг запускается, если пользователь выбрал "Получить текст судебного решения"
@bot.message_handler(content_types=["text"])
def send_decision(message):

  chat_id = message.chat.id

  # подчищаем директорию
  for i in os.listdir():
    if str(chat_id) in i:
      os.remove(i)

  # загружаем собранный корпус из текстов судебных решений Верховного суда Нидерландов по гражданским делам за 2025 год
  # как собирался корпус - см. отдельную тетрадку
  # в таблице - тексты судебных решений, их размер, лексическое разнообразие и ключевые слова
  data_frame_decisions = pd.read_csv('/content/df_corpus_full_key_ttr.csv')

  # далее загруженная таблица фильтруется по нескольким параметрам
  # первый параметр - размер текста
  # второй параметр - лексическое разнообразие текста (используется только для текстов среднего и большого размера,
  # поскольку для маленьких текстов он непоказателен; маленькие тексты решений Верховного суда обычно типовые и состоят из отсылки
  # к решением нижестоящих инстанций и указания на то, поддерживает Верховный суд эти решения или нет)

  # в первом дата-фрейме короткие судебные решения
  # в них количество лемм меньше 200, а длина текста, подсчитанная наивно через split, меньше 600
  # с длины 685 начинаются тексты среднего размера, в них количество лемм больше 200
  data_frame_filtered_1 = data_frame_decisions.loc[data_frame_decisions['length'] <= 685]

  # в дата-фреймах 2A и 2B - судебные решения среднего размера, в 2B с более разнообразной лексикой, чем в 2A
  data_frame_filtered_2 = data_frame_decisions.loc[data_frame_decisions['length'] > 685]
  data_frame_filtered_2_corr = data_frame_filtered_2.loc[data_frame_filtered_2['length'] <= 4597]
  min_t_2 = data_frame_filtered_2_corr['ttr'].min()
  max_t_2 = data_frame_filtered_2_corr['ttr'].max()
  threshold_t_2 = measure_ttr(min_t_2, max_t_2)

  data_frame_filtered_2A = data_frame_filtered_2_corr.loc[data_frame_filtered_2_corr['ttr'] < threshold_t_2]
  data_frame_filtered_2B = data_frame_filtered_2_corr.loc[data_frame_filtered_2_corr['ttr'] >= threshold_t_2]

  # в дата-фремах 3A и 3B - длинные судебные решения, 3B - более лексически разнообразные, 3A - менее
  data_frame_filtered_3 = data_frame_decisions.loc[data_frame_decisions['length'] > 4597]
  min_t_3 = data_frame_filtered_3['ttr'].min()
  max_t_3 = data_frame_filtered_3['ttr'].max()
  threshold_t_3 = measure_ttr(min_t_3, max_t_3)

  data_frame_filtered_3A = data_frame_filtered_3.loc[data_frame_filtered_3['ttr'] < threshold_t_3]
  data_frame_filtered_3B = data_frame_filtered_3.loc[data_frame_filtered_3['ttr'] >= threshold_t_3]

  # далее чат-бот реагирует на запрос пользователя:
  # с помощью функции to_do отбирается текст, который нужно отослать пользователю,
  # и затем текст отсылается (в части отсылки текста немного повторяется код,
  # но я решила спрятать в функцию to_do только самый большой кусок с отбором текстов)

  if message.text == '1':
    choose_text = to_do(chat_id, data_frame_filtered_1, dictionary_1)

    with open(str(chat_id) + f'{choose_text[1]}', 'w', encoding='utf-8') as des:
      print(choose_text[0], file = des)

    doc = open(str(chat_id) + f'{choose_text[1]}', 'r', encoding='utf-8')
    bot.send_document(message.chat.id, doc)
    doc.close()

  elif message.text == '2A':
    choose_text = to_do(chat_id, data_frame_filtered_2A, dictionary_2A)

    with open(str(chat_id) + f'{choose_text[1]}', 'w', encoding='utf-8') as des:
      print(choose_text[0], file = des)

    doc = open(str(chat_id) + f'{choose_text[1]}', 'r', encoding='utf-8')
    bot.send_document(message.chat.id, doc)
    doc.close()

  elif message.text == '2B':
    choose_text = to_do(chat_id, data_frame_filtered_2B, dictionary_2B)

    with open(str(chat_id) + f'{choose_text[1]}', 'w', encoding='utf-8') as des:
      print(choose_text[0], file = des)

    doc = open(str(chat_id) + f'{choose_text[1]}', 'r', encoding='utf-8')
    bot.send_document(message.chat.id, doc)
    doc.close()

  elif message.text == '3A':
    choose_text = to_do(chat_id, data_frame_filtered_3A, dictionary_3A)

    with open(str(chat_id) + f'{choose_text[1]}', 'w', encoding='utf-8') as des:
      print(choose_text[0], file = des)

    doc = open(str(chat_id) + f'{choose_text[1]}', 'r', encoding='utf-8')
    bot.send_document(message.chat.id, doc)
    doc.close()

  elif message.text == '3B':
    choose_text = to_do(chat_id, data_frame_filtered_3B, dictionary_3B)

    with open(str(chat_id) + f'{choose_text[1]}', 'w', encoding='utf-8') as des:
      print(choose_text[0], file = des)

    doc = open(str(chat_id) + f'{choose_text[1]}', 'r', encoding='utf-8')
    bot.send_document(message.chat.id, doc)
    doc.close()

# запускаем бот
bot.polling(none_stop=True, interval=0)